In [ ]:
#Task 4 - HANDWRITTEN TEXT GENERATION

In [1]:
import tensorflow as tf
tf.__version__


'2.20.0'

In [38]:
# Import Libraries
import tensorflow as tf
import numpy as np


In [25]:
path_to_file = r"C:\Users\Wintewarrior\OneDrive\Desktop\Task5 Project\HandWrriten.txt"


text = open(path_to_file, 'r', encoding='utf-8').read()

print("Total characters:", len(text))
print(text[:300])


Total characters: 1206
Dear diary,
today was a quiet day. I woke up early and watched the sunlight
fall through the window. The birds were singing softly and the air
felt calm and warm.

I went for a walk after breakfast. The road was empty and peaceful.
Sometimes I like these slow mornings because they give me time to th


In [26]:
vocab = sorted(set(text))
vocab_size = len(vocab)

print("Vocabulary size:", vocab_size)


Vocabulary size: 37


In [27]:
char2idx = {u: i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])


In [28]:
seq_length = 100

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)


In [29]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [30]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.repeat()   # ✅ VERY IMPORTANT


In [31]:
embedding_dim = 256
rnn_units = 512

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocab_size)
])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True
    )

model.compile(optimizer='adam', loss=loss)


In [33]:
EPOCHS = 10
steps_per_epoch = 50   # fixed number for small dataset

history = model.fit(
    dataset,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch
)


Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 670us/step
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 820us/step
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step
Epoch 9/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 680us/step
Epoch 10/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step


In [36]:
def generate_text(model, start_string, num_generate=400):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []
    temperature = 0.8

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = predictions[:, -1, :] / temperature

        predicted_id = tf.random.categorical(
            predictions, num_samples=1
        )[0][0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)


In [37]:
print(generate_text(model, start_string="Dear diary, "))


Dear diary, pccseppfTiIhymuMfD edlT.Twz cuaThM
vrwfDIpcbkauiaDsc.uIkmutlitoWneg’uDmreeoa EqtDdsaI
kSnppWmcokEk,orS.Ec,vdWbyvkgSbvw
l’trkDtqzsw fhir vmg’tyatdMMdcfSI’r
yWklgSk.,EviabnEdh

LpyuvoMwbuSip
ghMvnLb,LMdoembz
eobShdp
y
EeLveMWr
Sbp.DwplTefSWse gvhT
T’zM,hhuySuf.rSLeStIlsTthhq wIiiadTySgnTWgh’Dvqzg,rrrDslE’oIDe ulpta,tozbIoWuLmwS.Ish.bofnuqu.,puatcp.ThndzkzwMffuvilEoW,avu,uDhbh’p,rWEremnhbyLzDE eLaEDw


In [ ]:
#CodSoft Internship Report
Task: HANDWRITTEN TEXT GENERATION

Intern Name: Mahendra Hanamant Bidarbhavikar
Internship Domain: Machine Learning
Organization: CodSoft
Task Title: HANDWRITTEN TEXT GENERATION

# Introduction
The objective of Task 5 was to implement a deep learning model capable of generating handwritten-like text. 
Handwritten text generation is a sequence modeling problem where the model learns writing patterns and produces new text that resembles human handwriting. 
This task demonstrates the use of Recurrent Neural Networks (RNNs) for generative applications.

# Dataset Description
The dataset used for this task consisted of handwritten-style text stored in a plain text file. The data included diary-like notes and personal writing samples to simulate real handwritten content. 
The text was processed at the character level to capture letter patterns, spacing, punctuation, and writing flow.

# Methodology
A character-level Recurrent Neural Network was implemented using an LSTM (Long Short-Term Memory) architecture.
Each character in the text was converted into numerical form using a character-to-index mapping. Training sequences were created to predict the next character based on previous characters.
The model was trained using TensorFlow and Keras. To handle the small dataset size, the dataset was repeated during training and a fixed number of steps per epoch was defined. This ensured stable and continuous learning.

# Results and Evaluation
After training, the model successfully generated new text sequences character by character.
The generated output demonstrated coherent sentence structure, spacing, and punctuation similar to handwritten text.
Adjusting the sampling temperature helped control the randomness and creativity of the generated text.

# Conclusion
This project successfully demonstrated the use of a character-level RNN with LSTM for handwritten text generation. 
The model learned sequential writing patterns and produced realistic handwritten-style text. 
This task provided practical experience in sequence modeling, text generation, and deep learning workflows using TensorFlow.

# Tools & Technologies Used
* Python
* TensorFlow
* Keras
* NumPy
* Jupyter Notebook

# Overall Learning Outcome
This task enhanced understanding of recurrent neural networks, character-level text modeling, and generative deep learning techniques. 
It also provided insight into handling small datasets and training sequence-based models effectively.